In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter

In [3]:
RELATIVE_PATH = "../../data/landing/housing/"
RELATIVE_OUT = "../../data/raw/historic/"

### Functions

In [4]:
"""will basically create a record for each entry of one of the columns, and a new column for another entry
Note: assumes a multi index
Note: index_stop is the number of columns in the original dataframe that don't need to be melted"""
def unpack_levels(df, index_stop, level_1_name, level_2_name, record_level_2=False):
    # make a massive dataframe, everything is a record
    df = df.melt(id_vars=df.columns.to_list()[:index_stop], 
                 value_vars = df.columns.to_list()[index_stop:],
                 var_name=[level_1_name, level_2_name],
                 value_name="value")

    if (not record_level_2):
        # basically extract the column data from the second level
        df = df.pivot(index=list(df.columns[:index_stop]) + [level_1_name], 
                      columns=level_2_name, values="value")
        
    else:
        # switch around the melted columns so the end on gets melted
        df = df.pivot(index=list(df.columns[:index_stop]) + [level_2_name], 
                      columns=level_1_name, values="value")

    # reset the indices and rename
    df = df.reset_index()
    #print([column[0] for column in df.columns[:index_stop]])
    df.columns = [column[0] for column in df.columns[:index_stop]] + list(df.columns[index_stop:])

    return df


### Download

In [16]:
# get a list of dataframes to merge
housing_df_array = []
for file_name in os.listdir(RELATIVE_PATH):
    if (file_name != "all_properties.csv"):
        # get the information about the file
        curr_type = file_name.split("_")[0]
        curr_beds = file_name.split("_")[1]
    else:
        curr_type = "all"
        curr_beds = "all"

    # download the frame and add information
    curr_df = pd.read_csv(RELATIVE_PATH + file_name)
    curr_df["type"] = curr_type
    curr_df["beds"] = curr_beds

    # drop the first column
    curr_df = curr_df.drop(curr_df.columns[0], axis=1)

    # add the dataframe to the list
    housing_df_array.append(curr_df)

# merge the dataframes
housing_df_raw = pd.concat(housing_df_array)

print(housing_df_raw.shape)
print(housing_df_raw.dtypes)
housing_df_raw.head(5)

(1113, 199)
Unnamed: 1    object
Mar 2000      object
Mar 2000.1    object
Jun 2000      object
Jun 2000.1    object
               ...  
Mar 2024.1    object
Jun 2024      object
Jun 2024.1    object
type          object
beds          object
Length: 199, dtype: object


,Unnamed: 1,Mar 2000,Mar 2000.1,Jun 2000,Jun 2000.1,Sep 2000,Sep 2000.1,Dec 2000,Dec 2000.1,Mar 2001,...,Sep 2023,Sep 2023.1,Dec 2023,Dec 2023.1,Mar 2024,Mar 2024.1,Jun 2024,Jun 2024.1,type,beds
0,Albert Park-Middle Park-West St Kilda,172,300,156,310,151,320,148,320,147,...,71,700,75,720,78,738,71,740,house,2
1,Armadale,66,290,70,290,64,298,62,300,61,...,39,750,36,768,27,775,26,750,house,2
2,Carlton North,204,250,217,260,221,260,200,260,187,...,131,650,119,675,111,685,108,700,house,2
3,Carlton-Parkville,109,250,112,253,91,250,96,250,113,...,68,600,62,630,51,650,54,655,house,2
4,CBD-St Kilda Rd,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,house,2


- Manually checked each column followed this pattern
- They actually made a mistake on the dataset, 2003 dec replaced 2002 dec

### Renaming, Conversion and Missing values

In [17]:
housing_df_raw.columns[-3:]

Index(['Jun 2024.1', 'type', 'beds'], dtype='object')

In [18]:
# get the measure columns
time_stamps = [str(year) + "-" + month 
               for year in range(2000, 2025) 
               for month in ["03", "06", "09", "12"]]

# remove the last 2 quarters (because don't exist in the dataframe)
time_stamps = time_stamps[:-2]

measure_columns = [(time_stamp, measure) for time_stamp in time_stamps for measure in ["count", "median"]]

# rename the columns
housing_df_raw.columns = pd.MultiIndex.from_tuples([("region", None)] + measure_columns + [("type", None), ("beds", None)])

# reverse sort the column names
housing_df_raw = housing_df_raw.iloc[:, [0, -2, -1] + list(range(1, len(housing_df_raw.columns) - 2))]

In [19]:
# remove records with group total
housing_df_raw = housing_df_raw[~housing_df_raw[('region', None)].str.contains('Group Total')]

# Spelling correction
housing_df_raw['region'] = housing_df_raw['region'].replace('Wanagaratta', 'Wangaratta')
housing_df_raw['region'] = housing_df_raw['region'].replace('Newcombe', 'Newcomb')

# find the na values
housing_df_raw = housing_df_raw.replace("-", np.nan)

# convert all values to integers
for measure_column in housing_df_raw.columns[3:]:
    housing_df_raw[measure_column] = housing_df_raw[measure_column].astype(float)

housing_df_raw.head(5)

region   type beds 2000-03        2000-06  \
                                     NaN    NaN  NaN   count median   count   
0  Albert Park-Middle Park-West St Kilda  house    2   172.0  300.0   156.0   
1                               Armadale  house    2    66.0  290.0    70.0   
2                          Carlton North  house    2   204.0  250.0   217.0   
3                      Carlton-Parkville  house    2   109.0  250.0   112.0   
4                        CBD-St Kilda Rd  house    2     NaN    NaN     NaN   

         2000-09        2000-12  ... 2023-06        2023-09        2023-12  \
  median   count median   count  ...   count median   count median   count   
0  310.0   151.0  320.0   148.0  ...    73.0  680.0    71.0  700.0    75.0   
1  290.0    64.0  298.0    62.0  ...    46.0  698.0    39.0  750.0    36.0   
2  260.0   221.0  260.0   200.0  ...   132.0  640.0   131.0  650.0   119.0   
3  253.0    91.0  250.0    96.0  ...    72.0  590.0    68.0  600.0    62.0   
4    NaN     NaN    NaN     NaN  ...     NaN    NaN     NaN    NaN     NaN   

         2024-03        2024-06         
  median   count median   count median  
0  720.0    78.0  738.0    71.0  740.0  
1  768.0    27.0  775.0    26.0  750.0  
2  675.0   111.0  685.0   108.0  700.0  
3  630.0    51.0  650.0    54.0  655.0  
4    NaN     NaN    NaN     NaN    NaN  

[5 rows x 199 columns]

### Melting

In [20]:
# number of values want
N_VALUES = 7

def get_previous_periods(series):
    previous_values = []
    previous_value_array = []
    for i, value in series.items():
        # add the previous values
        previous_value_array.append(previous_values.copy())

        # update the seen values
        previous_values.append(value)

        # remove the first value if 
        if (len(previous_values) > N_VALUES):
            previous_values = previous_values[1:]

    # return the series
    return pd.Series(previous_value_array, index=series.index)

In [21]:
# turn everything into records
housing_df = unpack_levels(housing_df_raw, 3, "time stamp", "measure")

# create quarters and years
housing_df[["year", "quarter"]] = housing_df["time stamp"].str.split("-", expand=True).astype(int)

# get the actual quarter
housing_df["quarter"] = housing_df["quarter"] // 3

# get the previous 5 periods
housing_df["previous 2 years"] = housing_df.groupby("region", group_keys=False)["median"].apply(get_previous_periods)

# get the average for the time period
housing_df["avg 2 years"] = housing_df["previous 2 years"].apply(lambda x: sum(x) / len(x) if len(x) > 0 else None)

# to csv
housing_df.to_csv(RELATIVE_OUT + "housing.csv", index=False)

housing_df.head(10)

,region,type,beds,time stamp,count,median,year,quarter,previous 2 years,avg 2 years
0,Albert Park-Middle Park-West St Kilda,all,all,2000-03,1143.0,260.0,2000,1,[],NaN
1,Albert Park-Middle Park-West St Kilda,all,all,2000-06,1134.0,260.0,2000,2,[260.0],260.000000
2,Albert Park-Middle Park-West St Kilda,all,all,2000-09,1177.0,270.0,2000,3,"[260.0, 260.0]",260.000000
3,Albert Park-Middle Park-West St Kilda,all,all,2000-12,1178.0,275.0,2000,4,"[260.0, 260.0, 270.0]",263.333333
4,Albert Park-Middle Park-West St Kilda,all,all,2001-03,1208.0,275.0,2001,1,"[260.0, 260.0, 270.0, 275.0]",266.250000
5,Albert Park-Middle Park-West St Kilda,all,all,2001-06,1245.0,280.0,2001,2,"[260.0, 260.0, 270.0, 275.0, 275.0]",268.000000
6,Albert Park-Middle Park-West St Kilda,all,all,2001-09,1244.0,280.0,2001,3,"[260.0, 260.0, 270.0, 275.0, 275.0, 280.0]",270.000000
7,Albert Park-Middle Park-West St Kilda,all,all,2001-12,1293.0,290.0,2001,4,"[260.0, 260.0, 270.0, 275.0, 275.0, 280.0, 280.0]",271.428571
8,Albert Park-Middle Park-West St Kilda,all,all,2002-03,1332.0,300.0,2002,1,"[260.0, 270.0, 275.0, 275.0, 280.0, 280.0, 290.0]",275.714286
9,Albert Park-Middle Park-West St Kilda,all,all,2002-06,1350.0,300.0,2002,2,"[270.0, 275.0, 275.0, 280.0, 280.0, 290.0, 300.0]",281.428571


## Exploration and Checking stuff

### Checking how many null values will be deleted

So the below basically tests how many growth rate records will be deleted when calculating the yearly growth rate, only accepting years will MAX_THRES years or less

In [59]:
START = 3
MAX_THRES = 0


print(housing_median.shape[0] * 23)

# split into batches
for max_thres in range(3):
    sum_lost = 0

    for batch_i in range(int((len(housing_median.columns) - 1 - START) / 4)):
        subset = housing_median.iloc[:, START + 4*batch_i: START + 4*(batch_i+2)]
        # checking the amoun     t of times the amount of na's is exceeded for the previous or current year
        sum_lost += subset.isna().apply(lambda x: (x.iloc[:4].sum() > max_thres) | 
                                        (x.iloc[4:].sum() > max_thres), axis=1).sum()

    print(sum_lost)

19044
1061
860
708


### Blah